In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/optiver-trading-at-the-close.zip"

#Code for Generating New Features Being Engineered for Training the Model

In [ ]:
#UNCOMMENT AND RUN THIS CODE TO CREATE .CSV FILE WITH NEWLY GENERATED FEATURES FOR TRAINING THE MODEL

import pandas as pd

df = pd.read_csv("train.csv")

df.dropna(inplace=True)

df["combined_imbalance"] = df["imbalance_size"] * df["imbalance_buy_sell_flag"]
df["fn_price_diff"] = df["far_price"] - df["near_price"]
df["nr_price_diff"] = df["near_price"] - df["reference_price"]
df["fr_price_diff"] = df["far_price"] - df["reference_price"]
df["im_size_diff"] = df["imbalance_size"] - df["matched_size"]
df["imb_ratio"] = df["matched_size"] - df["imbalance_size"]
df["bid_ask_spread"] = (
    df["bid_price"] * df["bid_size"] - df["ask_price"] * df["ask_size"]
)

output_file_path = "train_with_new_features_new.csv"
df.to_csv(output_file_path, index=False)

print(f"File saved to {output_file_path}")


#Pre-Processing Code for TCN Models

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset


# df = pd.read_csv("train.csv") #For Without Feature Engineering
df = pd.read_csv("train_with_new_features_new.csv") #For With Feature Engineering


df.dropna(inplace=True)

features = df.drop(["target", "row_id", "time_id"], axis=1)
target = df["target"]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

num_features = features_scaled.shape[1]
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train, dtype=torch.float32).view(-1, num_features, 1)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).view(-1, num_features, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)


train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory = True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory = True)


#Basic M-TCN Model

In [ ]:
class UnitBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation_rates):
        super(UnitBlock, self).__init__()
        self.dilated_conv1 = nn.Conv1d(in_channels, out_channels, kernel_size, dilation=dilation_rates[0], padding=(kernel_size-1) * dilation_rates[0] // 2)
        self.dilated_conv2 = nn.Conv1d(in_channels, out_channels, kernel_size, dilation=dilation_rates[1], padding=(kernel_size-1) * dilation_rates[1] // 2)
        self.relu = nn.ReLU()

    def forward(self, x):
        out1 = self.relu(self.dilated_conv1(x))
        out2 = self.relu(self.dilated_conv2(x))
        return out1 + out2

class MTCN(nn.Module):
    def __init__(self, num_features, num_outputs, out_channels, kernel_size, max_dilation, num_blocks):
        super(MTCN, self).__init__()
        self.blocks = nn.ModuleList()
        current_dilation = 1
        for _ in range(num_blocks):
            dilation_rates = [current_dilation, current_dilation * 2]
            self.blocks.append(UnitBlock(num_features if _ == 0 else out_channels, out_channels, kernel_size, dilation_rates))
            current_dilation *= 4

        self.output_layer = nn.Linear(out_channels, num_outputs)

    def forward(self, x):
        for block in self.blocks:
            x = block(x)
        x = x.mean(dim=2)
        return self.output_layer(x)

num_features = features_scaled.shape[1]
kernel_size = 3
out_channels = 32
max_dilation = 8
num_blocks = 1

mtcn = MTCN(num_features, num_outputs=1, out_channels=out_channels, kernel_size=kernel_size, max_dilation=max_dilation, num_blocks=num_blocks)

#Residual M-TCN Model

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class DilatedConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, dilation_rate, padding):
        super(DilatedConvBlock, self).__init__()
        self.conv = nn.Conv1d(in_channels, out_channels, kernel_size, dilation=dilation_rate, padding=padding)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.relu(self.conv(x))

class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, num_units, max_dilation):
        super(ResidualBlock, self).__init__()
        self.units = nn.ModuleList()
        for i in range(num_units):
            dilation_rate = 2 ** i if i < max_dilation else 2 ** max_dilation
            padding = (kernel_size - 1) * dilation_rate // 2
            self.units.append(DilatedConvBlock(in_channels if i == 0 else out_channels, out_channels, kernel_size, dilation_rate, padding))

        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else nn.Identity()

    def forward(self, x):
        residual = self.downsample(x)
        for unit in self.units:
            x = unit(x)
        return x + residual

class MTCN(nn.Module):
    def __init__(self, num_features, num_outputs, out_channels, kernel_size, max_dilation):
        super(MTCN, self).__init__()
        self.heads = nn.ModuleList([
            nn.Sequential(
                ResidualBlock(1, out_channels, kernel_size, 3, max_dilation),
                ResidualBlock(out_channels, out_channels, kernel_size, 4, max_dilation),
                ResidualBlock(out_channels, out_channels, kernel_size, 3, max_dilation)
            ) for _ in range(num_features)
        ])
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(num_features * out_channels, num_outputs)

    def forward(self, x):
        outputs = []
        for i in range(x.size(1)):
            y = x[:, i:i+1, :]
            y = self.heads[i](y)
            outputs.append(self.flatten(y))
        x = torch.cat(outputs, dim=1)
        x = self.fc(x)
        return x

num_features = X_train_tensor.shape[1]
kernel_size = 3
out_channels = 32
max_dilation = 8
mtcn = MTCN(num_features, num_outputs=1, kernel_size=kernel_size, out_channels=out_channels, max_dilation=max_dilation)



#LSTM Model with the Necessary Pre-Processing

In [ ]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

# df = pd.read_csv("train.csv") #For Without Feature Engineering
df = pd.read_csv("train_with_new_features_new.csv") #For With Feature Engineering

print(df.head)

df.dropna(inplace=True)
print("Rows with NaN values have been dropped.")

features = df.drop(["target", "row_id", "time_id"], axis=1)
target = df["target"]

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

features_scaled = np.reshape(
    features_scaled, (features_scaled.shape[0], 1, features_scaled.shape[1])
)

X_train, X_test, y_train, y_test = train_test_split(
    features_scaled, target, test_size=0.2, random_state=42
)

model = Sequential()
model.add(
    LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]))
)
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

learning_rate = 0.0001

adam_optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=adam_optimizer, loss="mse")


for epoch in range(50):
    try:
        history = model.fit(
            X_train,
            y_train,
            epochs=1,
            batch_size=64,
            validation_data=(X_test, y_test),
            verbose=1,
        )
    except Exception as e:
        print(f"An error occurred in epoch {epoch+1}: {e}")
        continue


predicted = model.predict(X_test)
mse = mean_squared_error(y_test, predicted)
mae = mean_absolute_error(y_test, predicted)
rmse = np.sqrt(mse)

# Print the evaluation metrics
print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")
print(f"Test RMSE: {rmse:.4f}")

plt.figure(figsize=(10, 5))
plt.plot(history.history["loss"], label="Training Loss")
plt.title("Training Loss Over Epochs")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig("lstm1_lossplot_new_features_lr=0.0001.png")
plt.show()

model.save("lstm_model_new_features.h5")
